## Imports & Constants

In [28]:
import numpy as np
import pandas as pd
import numpy.typing as npt
from abc import ABC, abstractmethod
from sklearn.naive_bayes import MultinomialNB, GaussianNB

from dataset_types import ReviewDataSet
from feature_generation import FeatureSetGenerator
from feature_normalization import FeatureSetNormalizer

POSITIVE_REVIEWS_DIR = "./data/pos/"
NEGATIVE_REVIEWS_DIR = "./data/neg/"

## Data

In [5]:
dataset = ReviewDataSet([POSITIVE_REVIEWS_DIR, NEGATIVE_REVIEWS_DIR]).load()

In [6]:
feature_set = FeatureSetGenerator(dataset)\
    .remove_stopwords()\
    .remove_punctuation()\
    .lemmatize()\
    .create_n_grams(1)

In [7]:
normalizer = FeatureSetNormalizer(feature_set)
normalized_feature_set = normalizer.perform_fast_tf_idf()

In [8]:
X_train, y_train, X_dev, y_dev, X_test, y_test = normalized_feature_set.split_into_train_dev_test_sets("polarity", 0.3)

## Naive Bayes Classifier

Naive Bayes uses a 'bag of words approach', where individual words constitute its features, and the other of the words is ignored.
- The adjective 'Naive' connotes the asssumption that the occurence of features in a dataset are mutually independent.
    - In reality, this is a performance-damaging assumption

[Variants of Naive Bayes Classifiers](https://en.wikipedia.org/wiki/Naive_Bayes_classifier)
- With a `mutlinomial` event model, samples (feature vectors) the frequencies with which certain events
have been generated by a `multinomial` (p_1, ..., p_n) where p_i is the probability that event i occurs.
    - Or K such multinomials in the _multiclass_ case
    - This is the event model typically used for document classification (e.g., sentiment analysis)
        - Events can represent the occurence of a word in a single document
        - In our case, events represent the TF-IDF of words in a document
- `Guassian` distributions are typically used when dealing with continuous data
    - I gather `mutlinomial` is much better for NLP applications

### Helpful Definitions
**Prior Probability**: The probability of an event (e.g., a spam email) before the collection of new data.
- I.e., if prior observations are used to calculate the probability, we call it the prior probability

**Conditional Likelihoods**:

**Maximum a Posteriori**:

In [47]:
class IClassifier(ABC):

    @abstractmethod
    def fit(self):
        raise NotImplementedError

    @abstractmethod
    def predict(self):
        raise NotImplementedError

class NaiveBayesClassifier:

    def __init__(self, k: int =2, alpha: float = 1):
        """
        
        :param k: The number of classes in the classification problem.
        :param alpha: A laplace smoothing parameter.
        """
        self.k = k
        self.alpha = alpha

    def fit(self, data: npt.NDArray, labels: npt.NDArray):
        num_samples, num_features = data.shape

        # Allocate memory for the class prior probabilities and the class-conditioned feature likelihoods.
        self.class_prior_probas = np.zeros(shape=(self.k,))
        self.class_conditional_feature_likelihoods = np.zeros(shape=(self.k, num_features))

        for cls in range(self.k):
            samples_of_class = data[labels == cls]
            class_sample_count = samples_of_class.shape[0]

            # Probability class cls occurs. P(Y)
            self.class_prior_probas[cls] = class_sample_count / num_samples

            # Term frequencies for class `cls`.
            cls_global_feature_totals = samples_of_class.sum(axis=0) # shape = (num_features, )

            # Compute the conditional probability for each feature given the class
            # (including Laplace smoothing). P(X | Y)
            self.class_conditional_feature_likelihoods[cls, :] = (
                    (cls_global_feature_totals + self.alpha) /
                    (np.sum(cls_global_feature_totals) + self.alpha * num_features)
                )

    def legacy_predict(self, data: npt.NDArray):
        num_samples, _ = data.shape

        # Allocate memory for our predictions.
        predictions = np.zeros(shape=(num_samples,))

        # Calculate the log of class prior probabilities.
        log_class_priors = np.log(self.class_prior_probas)

        for idx, sample in enumerate(data):
            # Log probabilities of each class for the current sample.
            logits = np.zeros(shape=(self.k,))

            for cls in range(self.k):
                # The log probability of each feature given the class.
                feature_log_probabilities = sample * np.log(self.class_conditional_feature_likelihoods[cls, :])

                # Sum the feature log probabilties to get the total log probability of the sample
                # given the class.
                log_prob_features_given_class = np.sum(feature_log_probabilities)

                # Add the prior probability of the class itself to get the final log probability
                # for the class.
                logits[cls] = log_class_priors[cls] + log_prob_features_given_class

            predictions[idx] = np.argmax(logits)

        return predictions
    
    def fast_fit(self, data: npt.NDArray, labels: npt.NDArray):
        pass
    
    def predict(self, data: npt.NDArray) -> npt.NDArray:
        # Calculate the log of the priors.
        log_class_priors = np.log(self.class_prior_probas)

        # Calculate the log probabilities of each class for each sample.
        logits = log_class_priors + (data @ np.log(self.class_conditional_feature_likelihoods).T)

        # For each sample, get the index of the maximum logit. These are the predicted classes.
        return np.argmax(logits, axis=1)

## Evaluation Classes

In [56]:
class ClassifierPerformanceSummary:

    def __init__(self, accuracy: float, precision: float, recall: float, f1: float):
        self.accuracy = accuracy
        self.precision = precision
        self.recall = recall
        self.f1 = f1

    def __repr__(self) -> str:
        return f"""
        Accuracy:  {self.accuracy*100:.4f}%
        Precision: {self.precision*100:.4f}%
        Recall:    {self.recall*100:.4f}%
        F1:        {self.f1*100:.4f}%
        """
    
    def as_dict(self) -> dict:
        return {
            'accuracy': self.accuracy,
            'precision': self.precision,
            'recall': self.recall,
            'f1': self.f1
        }


class BinaryClassifierEvaluator:

    def __init__(self, ground_truth: npt.NDArray, predictions: npt.NDArray):
        self.predictions = predictions
        self.ground_truth = ground_truth
        self.confusion_matrix = self.produce_confusion_matrix()
        """
        [[ TP, FP ],
         [ FN, TN ]]
        """

    def produce_confusion_matrix(self) -> npt.NDArray:
        confusion_matrix = np.zeros(shape=(2, 2))

        # True positives: where both prediction and ground truth are 1.
        confusion_matrix[0, 0] = np.sum((self.predictions == 1) & (self.ground_truth == 1))

        # False positives: where prediction is 1, but ground truth is 0.
        confusion_matrix[0, 1] = np.sum((self.predictions == 1) & (self.ground_truth == 0))
        
        # False negatives: where prediction is 0, but ground truth is 1.
        confusion_matrix[1, 0] = np.sum((self.predictions == 0) & (self.ground_truth == 1))
        
        # True negatives: where both prediction and ground truth are 0.
        confusion_matrix[1, 1] = np.sum((self.predictions == 0) & (self.ground_truth == 0))

        return confusion_matrix.astype(int)

    def calculate_accuracy(self) -> float:
        """
        Calculates (TP + TN) / Num Predictions.
        """
        return (self.confusion_matrix[0, 0] + self.confusion_matrix[1, 1]) / self.confusion_matrix.sum()

    def calculate_recall(self) -> float:
        """
        Calculates TP / (TP + FN)
        """
        return self.confusion_matrix[0, 0] / self.confusion_matrix[:, 0].sum()

    def calculate_precision(self) -> float:
        """
        Calculates TP / (TP + FP).
        """
        return self.confusion_matrix[0, 0] / self.confusion_matrix[0, :].sum()
    
    def calculate_f1(self) -> float:
        """
        Calculates the harmonic mean of precision and recall.
        """
        precision = self.calculate_precision()
        recall = self.calculate_recall()

        return 2 * (precision * recall) / (precision + recall)
    
    def get_summary(self) -> ClassifierPerformanceSummary:
        return ClassifierPerformanceSummary(
            self.calculate_accuracy(),
            self.calculate_precision(),
            self.calculate_recall(),
            self.calculate_f1()
        )


def evaluate_feature_sets(feature_sets: list[tuple[npt.NDArray, npt.NDArray]], X_dev: npt.NDArray, y_dev: npt.NDArray, classifier_class: IClassifier, hyperparams: dict = {}):
    feature_set_performances = pd.DataFrame()
    
    for i, (data, labels) in enumerate(feature_sets):
        classifier = classifier_class(**hyperparams)
        classifier.fit(data, labels)

        predictions = classifier.predict(X_dev)
        performance = BinaryClassifierEvaluator(y_dev, predictions).get_summary().as_dict()
        feature_set_performances[f"{classifier_class.__name__} - Set {i}"] = performance

    return feature_set_performances.T


def evaluate_classifiers_over_feature_sets(classifier_classes: list[IClassifier], feature_sets: list[tuple[npt.NDArray, npt.NDArray]], X_dev: npt.NDArray, y_dev: npt.NDArray, hyperparams: dict = {}):
    classifier_performances = pd.DataFrame()

    for cls in classifier_classes:
        performance = evaluate_feature_sets(
            [(X_train, y_train), (X_train, y_train)],
            X_dev,
            y_dev,
            cls,
            hyperparams,
        )
        classifier_performances = pd.concat([classifier_performances, performance], axis=0)

    return classifier_performances

## Evaluating our Naive Bayes Classifier

In [9]:
classifier = NaiveBayesClassifier(2, 1)
classifier.fit(X_train, y_train)

In [10]:
cdfl = classifier.class_conditional_feature_likelihoods
priors = classifier.class_prior_probas

In [11]:
dev_set_predictions = classifier.predict(X_dev)
BinaryClassifierEvaluator(y_dev, dev_set_predictions).get_summary()


        Accuracy:  84.6667%
        Precision: 84.6667%
        Recall:    84.6667%
        F1:        84.6667%
        

## Evaluating Sklearn's Multinomial Naive Bayes Classifier

In [49]:
mnb_classifier = MultinomialNB(alpha=1, fit_prior=True, force_alpha=True)

mnb_classifier.fit(X_train, y_train)
sklearn_dev_set_predictions = mnb_classifier.predict(X_dev)

BinaryClassifierEvaluator(y_dev, sklearn_dev_set_predictions).get_summary()


        Accuracy:  84.6667%
        Precision: 84.6667%
        Recall:    84.6667%
        F1:        84.6667%
        

In [57]:
classifier_performances = evaluate_classifiers_over_feature_sets(
    [MultinomialNB, GaussianNB, NaiveBayesClassifier],
    [(X_train, y_train), (X_train, y_train)],
    X_dev,
    y_dev,
    {}
)

In [58]:
classifier_performances.sort_values(by="f1")

,accuracy,precision,recall,f1
GaussianNB - Set 0,0.646667,0.634969,0.690000,0.661342
GaussianNB - Set 1,0.646667,0.634969,0.690000,0.661342
MultinomialNB - Set 0,0.846667,0.846667,0.846667,0.846667
MultinomialNB - Set 1,0.846667,0.846667,0.846667,0.846667
NaiveBayesClassifier - Set 0,0.846667,0.846667,0.846667,0.846667
NaiveBayesClassifier - Set 1,0.846667,0.846667,0.846667,0.846667
